In [1]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd
import os

import yaml
import hashlib

In [11]:
N = sp.symbols("N", integer=True)
t = sp.symbols('t')
_N = 2

γ = sp.symbols('γ')
E0, L, w1, σc = sp.symbols('E0 L w1 σc')

u = sp.symbols([f"u{i}" for i in range(0, _N)])
e = sp.symbols([f"e{i}" for i in range(1, _N+1)])
α = sp.symbols([f"α{i}" for i in range(1, _N+1)])
_α = sp.symbols("α")



In [3]:
state = {"u": u, "α": α, "e": e}
_matpar = {N: _N, γ: 2, "E0": 1, "w1": 1, "L": 1}
matpar = {"N": N, "γ": γ, "E0": E0, "w1": w1, "L": L}



In [4]:
_a = sp.Function('a')
_s = sp.Function('s')


In [12]:
_s = 1/_a(_α)

In [13]:
_s

1/a(α)

In [16]:
_ap = sp.diff(_a(_α), _α)

In [19]:
_ap.replace(_a, lambda x: ((1-x))**(2) ).doit()

2*α - 2

In [25]:
_ = sum([1./2. * E0 * N * _a(α[i]) * e[i]**2 for i in range(len(α))])

In [27]:
sp.diff(_, α[1]).replace(_a, lambda x: ((1-x))**(2) ).doit()

0.5*E0*N*e2**2*(2*α2 - 2)

In [64]:
class SymbolicDiscreteDamage:
    def __init__(self, state, matpar, 
                 name = "discrete generalised damage",
                 slug = "general damage"):
        self.state = state
        self.matpar = matpar
        self.str = name
        self.slug = slug

    def _s(self, αi):
        s = sp.Function('s')
        # s = 1/self._a(αi)
        return s(αi)

    def _a(self, αi):
        γ = self.matpar["γ"]
        a = sp.Function('a')
        return a(αi)
    
    def energy(self, state):
        """Total energy"""
        return self._elastic(state) + self._damage(state) - self.work(state)

    def work(self, state):
        return 0

    def _elastic(self, state):
        _e = state["e"]
        _α = state["α"]
        # _w1 = matpar["w1"]
        _E0 = self.matpar["E0"]
        _N = self.matpar["N"]
    
        return sum([1./2. * E0 * N * self._a(α[i]) * e[i]**2 for i in range(len(α))])

    def _w(self, αi):
        w = sp.Function('w')

        return w(αi)

    def _damage(self, state):
        _α = state["α"]
        _w1 = self.matpar["w1"]

        _N = self.matpar["N"]

        return sum([_w1 / _N * self._w(_α[i]) for i in range(len(_α))]) 
        # return _w1 * self._w(_α)

    def __str__(self):
     return self.str


In [65]:
atls = SymbolicDiscreteDamage(state, matpar)


In [68]:
atls._s

<bound method SymbolicDiscreteDamage._s of <__main__.SymbolicDiscreteDamage object at 0x12dcc9430>>

In [61]:
atls._s(α[1])

s(α2)

In [62]:
atls._elastic(state).replace(_a, lambda x: ((1-x))**(2) ).doit()

0.5*E0*N*e1**2*(1 - α1)**2 + 0.5*E0*N*e2**2*(1 - α2)**2

In [63]:
atls.energy(state)

0.5*E0*N*e1**2*a(α1) + 0.5*E0*N*e2**2*a(α2) + w1*w(α1)/N + w1*w(α2)/N

In [81]:

class ModelAnalysis:
    """Helper class to analyse stability properties of a model"""

    def __init__(self, model):
        self.model = model
        self.state = model.state
        _state = self.state
        self.matpar = model.matpar

        _β = sp.symbols('β')
        _e = sp.symbols('e')
        self._β = _β
        self._e = _e
        
        self._s = model._s(_β)
        self._sp = sp.diff(self._s, _β, 1)
        self._spp = sp.diff(self._s, _β, 2)

        self._a = model._a(_β)
        self._ap = sp.diff(self._a, _β, 1)
        self._app = sp.diff(self._a, _β, 2)

        self._w = model._w(_β)
        self._wp = sp.diff(self._w, _β)
        self._wpp = sp.diff(self._w, _β, 2)

        # self.criterion()


    def criterion(self):
        """Damage onset"""
        model, state = self.model, self.state
        x = sp.symbols('x')

        β = self._β

        _u0 = t*x/L
        _alpha0 = 0

        _homogeneous_damage = {a: β for a in state["α"]}
        _homogeneous_strain = {a: e for a in state["e"]}
        _sound = {a: 0 for a in state["α"]}

        _crit = sp.diff(model.energy(state), α) \
            .subs({u: _u0, α: _alpha0})         \
            .simplify()
        # .subs(matpar).simplify()
        # _ts = np.array(sp.solve(_crit, t))
        # return np.where(_ts > 0)
        self.crit = _crit

        return _crit


In [75]:
ModelAnalysis(atls)

In [85]:
β = sp.symbols('β')
_homogeneous = {a: β for a in state["α"]}
_sound = {a: 0 for a in state["α"]}



In [79]:
{a: 0 for a in state["α"]}

{α1: 0, α2: 0}

In [86]:
atls.energy(state).subs(_homogeneous)

0.5*E0*N*e1**2*a(β) + 0.5*E0*N*e2**2*a(β) + 2*w1*w(β)/N

In [71]:
sp.diff(atls.energy(state), α)

[α1, α2]